In [1]:
from writers import foyer_xml_writer
from writers.foyer_xml_writer import parmed_to_foyer_xml, mbuild_to_foyer_xml
from bondwalk import bond_walk
from bondwalk.bond_walk import MadAtom, MadBond, BondWalker
from mbuild.lib.recipes import Polymer
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield
from rdkit import Chem
from openff.toolkit.topology import Molecule

import ele
import espaloma as esp
import forcefield_utilities as ffutils
import foyer
import gmso
import gsd.hoomd
import hoomd
import matplotlib.pyplot as plt
import mbuild as mb
import networkx  as nx
import os
import parmed as pmd
import numpy as np
import torch
import rdkit
import warnings
warnings.filterwarnings("ignore")

if not os.path.exists("espaloma_model.pt"):
    os.system("wget http://data.wangyq.net/espaloma_model.pt")

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
The OpenEye Toolkits are found to be installed but not licensed and therefore will not be used.
The OpenEye Toolkits require a (free for academics) license, see https://docs.eyesopen.com/toolkits/python/quickstart-python/license.html
LICENSE: No product keys!
--2023-06-16 16:15:05--  http://data.wangyq.net/espaloma_model.pt
Resolving data.wangyq.net (data.wangyq.net)... 35.168.187.155, 52.2.56.64, 52.21.33.16
Connecting to data.wangyq.net (data.wangyq.net)|35.168.187.155|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://data.wangyq.net/espaloma_model.pt [following]
--2023-06-16 16:15:06--  https://data.wangyq.net/espaloma_model.pt
Connecting to data.wangyq.net (data.wangyq.net)|35.1

In [3]:
## creates the openff molecule object as well as saving a pdb file for renaming later
def openff_Molecule(mol2_file):
    rdmol = Chem.MolFromMol2File(mol2file)
    from_rdmol = Molecule.from_rdkit(rdmol)
    atom_list = [a for a in from_rdmol.atoms if 1 not in [a.atomic_number]]
    for atom in atom_list:
        atom.formal_charge = 0
    b = BondWalker(from_rdmol)
    comp = b.fill_in_bonds()
    from_rdmol.to_file(pdb_file,file_format="pdb")
    return comp

def getPairsBAD(openmm_system):
    return openmm_system.getForces()[1], openmm_system.getForces()[2], openmm_system.getForces()[3], openmm_system.getForces()[0]


In [13]:
#setup 
#molecule = openff_Molecule(mol2_file=mol2file)
pdb_file = "./pdbfiles/P3HT.pdb"
mol2file = './mol2files/P3HT_dimer.mol2'
xml_filename = "./xml_files/p3ht_md.xml"
typed_mol2 = "./mol2files/p3ht_md_typed.pdb"

molecule = openff_Molecule(mol2_file='mol2files/P3HT.mol2')
molecule_graph = esp.Graph(molecule)
espaloma_model = torch.load("espaloma_model.pt")
espaloma_model(molecule_graph.heterograph)
openmm_system = esp.graphs.deploy.openmm_system_from_graph(molecule_graph,charge_method="nn")

RadicalsNotSupportedError: The OpenFF Toolkit does not currently support parsing molecules with S- and P-block radicals. Found 1 radical electrons on molecule [H][C]1S[C]([C]2S[C]([H])[C](C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])[C]2[H])[C](C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])C([H])([H])[H])[C]1[H].

In [ ]:
pair_forces, bond_forces, angle_forces, torsion_forces = getPairsBAD(openmm_system)
topology = molecule.to_topology()
openmm_topology = topology.to_openmm()
structure = pmd.openmm.load_topology(topology=openmm_topology, system=openmm_system)
structure.bonds.sort(key=lambda x: x.atom1.idx)
for i in range(len(molecule.atoms)):
    molecule.atom[i].name = ele.element_from_atomic_number(molecule.atoms[i].atomic_number).symbol
Gopenmm = nx.Graph()
Gparmed = nx.Graph()
for i in range(bond_forces.getNumBonds()):#openmm:
    Gopenmm.add_edge(bond_forces.getBondParameters(index=i)[0],bond_forces.getBondParameters(index=i)[1])
for b in structure.bonds:#parmed
    Gparmed.add_edge(b.atom1.idx,b.atom2.idx)
    

In [2]:


#pair_forces = openmm_system.getForces()[1]
#angle_forces = openmm_system.getForces()[3]
#bond_forces = openmm_system.getForces()[2]
#torsion_forces = openmm_system.getForces()[0]

# get a parmed structure from openmm 






particle_types = []
type_map = dict()

#nx.rooted_tree_isomorphism
#in here we still need to check that one known index on one corresponds to the same index on the other....
tree_openmm = nx.bfs_tree(Gopenmm,0)
tree_parmed = nx.bfs_tree(Gparmed,0)
if nx.is_isomorphic(Gopenmm,Gparmed):
#if nx.isomorphism.tree_isomorphism(tree_openmm,tree_parmed):  <- want this work
    for i in range(pair_forces.getNumParticles()):
        pair_parms = pair_forces.getParticleParameters(index=i)
        sigma = pair_parms[1]/pair_parms[1].unit
        epsilon = pair_parms[2]/pair_parms[2].unit
        if (sigma, epsilon) not in particle_types: 
            particle_types.append((sigma, epsilon))
        type_map[molecule.atoms[i].molecule_atom_index] = "".join([molecule.atoms[i].name , 
                                                                   str(particle_types.index((sigma, epsilon)))])
        
# Rename the particle types so that they match the xml file
# This is needed when we aren't using SMARTS matching with Foyer.

comp_rename = mb.load(pdb_file)

for index in type_map:
    #print(index, type_map[index],comp_rename[index].name)
    comp_rename[index].name = type_map[index]

    

#Create your dictionaries:










bond_types = []
bond_dict = dict() 

for i in range(bond_forces.getNumBonds()):
    bond_parms = bond_forces.getBondParameters(index=i)
    l0 = bond_parms[2]/bond_parms[2].unit
    k = bond_parms[3]/bond_parms[3].unit
    bond_dict[type_map[bond_parms[0]],type_map[bond_parms[1]]] = {'k':k,'l0':l0}


angle_types = []
angle_dict = dict()

for i in range(angle_forces.getNumAngles()):
    angle_parms = angle_forces.getAngleParameters(index=i)
    k = angle_parms[4]/angle_parms[4].unit
    t0 = angle_parms[3]/angle_parms[3].unit  
    angle_dict[type_map[angle_parms[0]],type_map[angle_parms[1]],type_map[angle_parms[2]]] = {'k':k,'t0':t0}

dihedral_types = []
dihedral_dict = {}

for i in range(torsion_forces.getNumTorsions()):
    if i%6==0:
        periodicity=[]
        phase = []
        k = []
    dihedral_parms = torsion_forces.getTorsionParameters(index=i)
    periodicity.append(dihedral_parms[4])  
    phase.append( dihedral_parms[5]/dihedral_parms[5].unit)
    k.append(dihedral_parms[6]/dihedral_parms[6].unit)
    dt = (type_map[dihedral_parms[0]],type_map[dihedral_parms[1]],type_map[dihedral_parms[2]],
                  type_map[dihedral_parms[3]])
   

    if periodicity[-1]==6:
        dihedral_dict[dt] = {'periodicity':periodicity,'k':k,'phase':phase}


nonbonded_types = []
nonbonded_dict = {}

for i in range(pair_forces.getNumParticles()):
    nonbonded_parms = pair_forces.getParticleParameters(index=i)
    charge = nonbonded_parms[0]/nonbonded_parms[0].unit
    sigma = nonbonded_parms[1]/nonbonded_parms[1].unit
    epsilon = nonbonded_parms[2]/nonbonded_parms[2].unit
    nonbonded_types.append((charge,sigma,epsilon))
    nonbonded_dict[(type_map[i])]={'charge':charge,'sigma':sigma,'epsilon':epsilon}

LICENSE: Could not open license file "oe_license.txt" in local directory
LICENSE: N.B. OE_LICENSE environment variable is not set
LICENSE: N.B. OE_DIR environment variable is not set
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!
LICENSE: No product keys!


ImportError: cannot import name 'Molecule' from 'openff.toolkit' (/Users/ericjankowski/anaconda3/envs/ff2/lib/python3.9/site-packages/openff/toolkit/__init__.py)

In [ ]:
# Save the forcefield XML file for future use, so that we don't have to repeat the espaloma process everytime
mbuild_to_foyer_xml(
    file_name=xml_filename, #change this to whatever you want to save your xml file as
    compound=comp_rename,
    bond_params=bond_dict,
    angle_params=angle_dict,
    dihedral_params=dihedral_dict,
    dihedral_type="periodic",
    non_bonded_params=nonbonded_dict,
    combining_rule="geometric",
    name="",
    version="",
    coulomb14scale=1.0,
    lj14scale=1.0)

#Save the mb.Compound with the new atom type names for future use.
comp_rename.save(typed_mol2, overwrite=True) #change this to match your molecule name. 